In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from Bio import SeqIO

In [4]:
def loadsequencing(file):
    seq = SeqIO.read(file, 'abi')
    
    maxq = np.max(seq.letter_annotations['phred_quality'])
    rolling = pd.Series(seq.letter_annotations['phred_quality']).rolling(window=20).mean()
    start = (rolling > maxq * 0.9).idxmax()
    end = (rolling > maxq * 0.9)[::-1].idxmax()
    
    return seq.seq[start:end], seq, start, end, np.mean(seq.letter_annotations['phred_quality'][start:end])

In [5]:
plt.plot(seq.letter_annotations['phred_quality'])
plt.plot(pd.Series(seq.letter_annotations['phred_quality']).rolling(window=20).mean())
plt.axvline(x=start, c='g')
plt.axvline(x=end, c='r')

NameError: name 'seq' is not defined

In [6]:
SYNTH_DIRECTORY = "../synth1/"
BUILD_DIRECTORY = SYNTH_DIRECTORY + "builds/build-1/"

In [7]:
validation = []
twist = pd.read_csv('../synth1/backup/fragments.csv')

fragments = pd.DataFrame({
    'Gene': twist['Sequence ID'].str.rsplit('_', n=1).str[0],
    'Fragment': twist['Sequence ID'].str.rsplit('_', n=1).str[1],
    'Sequence': twist['Insert Sequence']
})

# Process single gene fragments
for i, gene in fragments[~fragments['Gene'].str.contains('_link_')].groupby('Gene'):
    frags = []
    for seq in gene['Sequence'].str.upper():
        # Virtual digest using BbsI
        seq = seq[seq.find('GAAGAC')+8:]
        seq = seq[:seq.find('GTCTTC')-2]
        frags.append(seq)

    # Now we have a list of 'digested' fragments; assemble them based on overlaps.
    while len(frags) > 1:
        frag = frags.pop(0)
        for other in frags:
            if other[-4:] == frag[:4]:
                frag = other[:-4] + frag
                frags.remove(other)
                break
        frags.append(frag)

    # Add assembled full sequence to the validation database
    # Remove the first and last nucleotides to get the CDS instead of MoClo adapters
    validation.append({'Gene': gene['Gene'].iloc[0], 'Sequence': frags[0][1:-1]})

# Process multigene fragments
for i, gene in fragments[fragments['Gene'].str.contains('_link_')].iterrows():
    names = gene['Gene'].split('_link_')

    # Virtual digest using BbsI
    seq = gene['Sequence'].upper()
    seq = seq[seq.find('GAAGAC')+8:]
    seq = seq[:seq.find('GTCTTC')-2]
    validation.append({'Gene': names[0], 'Sequence': seq[1:-1]})

    # We have a multi-gene fragment, digest with BtgZI
    seq = gene['Sequence'].upper()
    seq = seq[seq.find('GCGATG')+16:]
    seq = seq[:seq.find('CATCGC')-10]
    validation.append({'Gene': names[1], 'Sequence': seq[1:-1]})
    
validation_dna = pd.DataFrame(validation)

In [8]:
validation_dna.to_csv(SYNTH_DIRECTORY + "optimized.csv")

In [15]:
import glob
import os
import re

from Bio import pairwise2
from Bio import AlignIO
from Bio import Align
from Bio import Seq
from Bio import SeqRecord
from Bio.Alphabet import generic_dna

forward_primer = "M13-Forward---20-"
reverse_primer = "M13-Reverse"

all_alignments = []
bad_alignments = []
good_name = []
good_well = []

rfp_name = []
rfp_well = []

good = 0
rfp = 0

backbone_seq = SeqIO.read('./popen_v1-1_backbone.fasta', 'fasta')
backbone_seq = backbone_seq.seq

for seqfile in glob.glob(BUILD_DIRECTORY + "sequencing/*{}*.ab1".format(forward_primer)):
    
    goodF = 0
    goodR = 0
    rfpF = 0
    rfpR = 0

    print("Analysing sequence", seqfile)
    
    initials, order_number, plate_number, well_number, sample_name, primer_name, well_address = re.match(
        r'.*/([A-Z]+)_([0-9]+)-([0-9])([0-9]+)_([A-Za-z0-9_-]+)_([A-Za-z0-9-]+)_([A-H][0-9]{2}).ab1',
        seqfile).groups()
    revfile = "{}/{}_{}-{}{}_{}_{}_{}.ab1".format(os.path.dirname(seqfile), initials, order_number, (int(plate_number)+1), well_number, sample_name, reverse_primer, well_address)
    
    print("Validating {} ({}-{})".format(sample_name, plate_number, well_address))
    
    # We assume that for every forward there is a reverse
    forward, _, _, _, forward_qual = loadsequencing(seqfile)
    reverse, revseq, _, _, reverse_qual = loadsequencing(revfile)
    
    print("Quality", forward_qual, reverse_qual)
    
#     sample_name = re.sub(r'-*(.*)_[0-9]+', r'\1', sample_name)
    if "link" in sample_name:
        sample_name = re.sub(r'(.*)_link_.*', r'\1', sample_name)
        
    target_seq = validation_dna.loc[validation_dna['Gene']==sample_name, 'Sequence'].iloc[0]
        
    forward_untrim = forward
    reverse_untrim = reverse.reverse_complement()
    
    forward = forward[forward.find('ATG'):] # Start at start codon
    reverse = reverse[reverse.find('TCA'):].reverse_complement() # Stop at stop    
    
    forward_align = pairwise2.align.globalxx(target_seq, forward, one_alignment_only=True)
    reverse_align = pairwise2.align.globalxx(target_seq, reverse, one_alignment_only=True)
    
    if len(forward_align) == 0 or len(reverse_align) == 0:
        if len(forward_align) == 0:
            print("Forward doesn't align")

        if len(reverse_align) == 0:
            print("Reverse doesn't align")
            
        continue
    
    forward_align = forward_align[0]
    reverse_align = reverse_align[0]
    
    if (forward_align[2] != min(len(forward), len(target_seq))):
        print("Forward doesn't match gene: scores are {} and {}".format(forward_align[2],len(forward)))
        forward_align_back = pairwise2.align.globalxx(backbone_seq, forward_untrim, one_alignment_only=True)
        forward_align_back = forward_align_back[0]
        
        if (forward_align_back[2] == min(len(forward_untrim), len(backbone_seq))):
            print("Forward aligns to backbone: scores are {} and {}".format(forward_align_back[2],len(forward_untrim)))
            rfpF = rfpF + 1
        else:
            print("Forward doesn't align to backbone: scores are {} and {}".format(forward_align_back[2],len(forward_untrim)))
            
        align = Align.MultipleSeqAlignment([
            SeqRecord.SeqRecord(Seq.Seq(forward_align[0], generic_dna), id="TargetSequence"),
            SeqRecord.SeqRecord(Seq.Seq(forward_align[1], generic_dna), id="ForwardSequence"),
        ])
        bad_alignments.append(align)
    else:
        print("Forward matches gene: scores are {} and {}".format(forward_align[2],len(forward)))
        goodF = goodF + 1
        
    if (reverse_align[2] != min(len(reverse), len(target_seq))):
        print("Reverse doesn't match gene: scores are {} and {}".format(reverse_align[2],len(reverse)))
        reverse_align_back = pairwise2.align.globalxx(backbone_seq, reverse_untrim, one_alignment_only=True)
        reverse_align_back = reverse_align_back[0]

        if (reverse_align_back[2] == min(len(reverse_untrim), len(backbone_seq))):
            print("Reverse aligns to backbone: scores are {} and {}".format(reverse_align_back[2],len(reverse_untrim)))
            rfpR = 1
        else:
            print("Reverse doesn't align to backbone: scores are {} and {}".format(reverse_align_back[2],len(reverse_untrim)))            
        align = Align.MultipleSeqAlignment([
            SeqRecord.SeqRecord(Seq.Seq(reverse_align[0], generic_dna), id="TargetSequence"),
            SeqRecord.SeqRecord(Seq.Seq(reverse_align[1], generic_dna), id="ReverseSequence"),
        ])
        bad_alignments.append(align)
    else:
        print("Reverse matches gene: scores are {} and {}".format(reverse_align[2],len(reverse)))
        goodR = 1
    
    if goodF + goodR == 2:
        good = good + 1
        print(good)
        good_name.append(sample_name)
        good_well.append(well_address)
    if rfpF + rfpR == 2:
        rfp = rfp + 1
        print(rfp)
        rfp_name.append(sample_name)
        rfp_well.append(well_address)

    print()

print("good", good)
print("rfp",rfp)
good = pd.DataFrame({'Gene': good_name,
                     'Well': good_well})

rfp = pd.DataFrame({'Gene': rfp_name,
                    'Well': rfp_well})

AlignIO.write(bad_alignments, BUILD_DIRECTORY + 'bad-alignments.aln'.format(initials, order_number), 'clustal')

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1001_MMSYN1_0115_M13-Forward---20-_A01.ab1
Validating MMSYN1_0115 (1-A01)
Quality 60.0085034014 60.8630952381
Forward matches gene: scores are 542.0 and 542
Reverse matches gene: scores are 475.0 and 475
1

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1002_MMSYN1_0511_M13-Forward---20-_B01.ab1
Validating MMSYN1_0511 (1-B01)
Quality 60.3409893993 59.5457685665
Forward doesn't match gene: scores are 354.0 and 520
Forward aligns to backbone: scores are 566.0 and 566
Reverse doesn't match gene: scores are 370.0 and 540
Reverse aligns to backbone: scores are 579.0 and 579
1

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1003_MMSYN1_0215_M13-Forward---20-_C01.ab1
Validating MMSYN1_0215 (1-C01)
Quality 60.3360128617 60.423670669
Forward doesn't match gene: scores are 313.0 and 572
Forward aligns to backbone: scores are 622.0 and 622
Reverse doesn't match gene: scores are 311.0 and 540
Rever

Reverse aligns to backbone: scores are 619.0 and 619
17

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1024_MMSYN1_0045_M13-Forward---20-_H03.ab1
Validating MMSYN1_0045 (1-H03)
Quality 53.1596858639 60.424137931
Forward doesn't match gene: scores are 283.0 and 335
Forward aligns to backbone: scores are 382.0 and 382
Reverse doesn't match gene: scores are 372.0 and 540
Reverse aligns to backbone: scores are 580.0 and 580
18

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1025_MMSYN1_0283_M13-Forward---20-_A04.ab1
Validating MMSYN1_0283 (1-A04)
Quality 60.498392283 60.1590493601
Forward doesn't match gene: scores are 376.0 and 576
Forward aligns to backbone: scores are 622.0 and 622
Reverse doesn't match gene: scores are 357.0 and 513
Reverse aligns to backbone: scores are 547.0 and 547
19

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1026_MMSYN1_0416_M13-Forward---20-_B04.ab1
Validating MMSYN1_0416 (1-B04)
Quality 59.8852988691 60

Forward aligns to backbone: scores are 674.0 and 674
Reverse doesn't match gene: scores are 382.0 and 517
Reverse aligns to backbone: scores are 557.0 and 557
35

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1047_MMSYN1_0302_M13-Forward---20-_G06.ab1
Validating MMSYN1_0302 (1-G06)
Quality 59.9919224556 58.6724470135
Forward doesn't match gene: scores are 395.0 and 572
Forward aligns to backbone: scores are 619.0 and 619
Reverse doesn't match gene: scores are 368.0 and 511
Reverse aligns to backbone: scores are 519.0 and 519
36

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1048_MMSYN1_0614_M13-Forward---20-_H06.ab1
Validating MMSYN1_0614 (1-H06)
Quality 60.2656 60.1256454389
Forward doesn't match gene: scores are 481.0 and 574
Forward aligns to backbone: scores are 625.0 and 625
Reverse doesn't match gene: scores are 459.0 and 541
Reverse aligns to backbone: scores are 581.0 and 581
37

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525

Forward doesn't match gene: scores are 279.0 and 521
Forward aligns to backbone: scores are 559.0 and 559
Reverse doesn't match gene: scores are 284.0 and 517
Reverse aligns to backbone: scores are 522.0 and 522
53

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1070_MMSYN1_0338_M13-Forward---20-_F09.ab1
Validating MMSYN1_0338 (1-F09)
Quality 59.5445859873 58.1513292434
Forward doesn't match gene: scores are 409.0 and 576
Forward aligns to backbone: scores are 628.0 and 628
Reverse doesn't match gene: scores are 357.0 and 460
Reverse aligns to backbone: scores are 489.0 and 489
54

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1071_MMSYN1_0164_M13-Forward---20-_G09.ab1
Validating MMSYN1_0164 (1-G09)
Quality 60.4520325203 60.4493996569
Forward matches gene: scores are 495.0 and 563
Reverse matches gene: scores are 495.0 and 536
12

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1072_MMSYN1_0409_M13-Forward---20-_H09.ab1
Validating M

Forward doesn't match gene: scores are 359.0 and 574
Forward aligns to backbone: scores are 626.0 and 626
Reverse doesn't match gene: scores are 355.0 and 566
Reverse aligns to backbone: scores are 606.0 and 606
72

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1093_MMSYN1_0408_M13-Forward---20-_E12.ab1
Validating MMSYN1_0408 (1-E12)
Quality 60.1952 60.5211505922
Forward doesn't match gene: scores are 394.0 and 574
Forward aligns to backbone: scores are 625.0 and 625
Reverse doesn't match gene: scores are 381.0 and 543
Reverse aligns to backbone: scores are 591.0 and 591
73

Analysing sequence ../synth1/builds/build-1/sequencing/CM_525055-1094_MMSYN1_0094_M13-Forward---20-_F12.ab1
Validating MMSYN1_0094 (1-F12)
Quality 59.9255121043 59.4353146853
Forward doesn't match gene: scores are 373.0 and 492
Forward aligns to backbone: scores are 537.0 and 537
Reverse doesn't match gene: scores are 392.0 and 539
Reverse aligns to backbone: scores are 572.0 and 572
74

Analysin

157

In [17]:
good

,Gene,Well
0,MMSYN1_0115,A01
1,MMSYN1_0693,D01
2,MMSYN1_0346,F02
3,MMSYN1_0043,F03
4,MMSYN1_0686,D04
5,MMSYN1_0066,C05
6,MMSYN1_0326,F05
7,MMSYN1_0077,C06
8,MMSYN1_0440,A07
9,MMSYN1_0906,C07


In [ ]:
%%HTML
<script src=//cdn.bio.sh/msa/1.0/msa.min.gz.js></script>

In [ ]:
bad_alignments[0].format('clustal')

In [ ]:
from IPython.display import HTML
    
base = HTML('''
<script src=//cdn.bio.sh/msa/1.0/msa.min.gz.js></script>

<div id="alignment"></div>

<script type="text/Javascript">
var input = `''' + bad_alignments[0].format('clustal') + '''`;
var seqs = msa.io.clustal.parse(input);

var opts = {
    el: document.getElementById("alignment"),
    vis: {
      conserv: false,
      overviewbox: false
    },
    // smaller menu for JSBin
    menu: "small",
    bootstrapMenu: false,
    seqs: seqs
};

var m = new msa.msa(opts);
m.render();
</script>''')we

from IPython import display
display.display(base)

for align in bad_alignments:
    code = HTML('''
        <script type="text/Javascript">
            m.seqs = msa.io.clustal.parse(`{}`);
            m.render();
        </script>
    '''.format(align.format('clustal')))
    display.clear_output(wait=True)
    display.display((base, code))
    break